## Q1. Embedding the query

In [2]:
from fastembed import TextEmbedding
import numpy as np

In [ ]:
query = 'I just discovered the course. Can I join now?'
model_handle = "jinaai/jina-embeddings-v2-small-en"

embedder = TextEmbedding(model_name=model_handle)
embedding = next(embedder.embed(query))

embedding_array = np.array(embedding)
print("Minimal value:", embedding_array.min())

Fetching 5 files: 100%|██████████| 5/5 [00:01<00:00,  3.75it/s]


Minimal value: -0.11726373885183883


## Q2. Cosine similarity with another vector

In [4]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
from fastembed.embedding import TextEmbedding

# Initialize the embedder
embedder = TextEmbedding(model_name="jinaai/jina-embeddings-v2-small-en")

# Define query and document
query = ["I just discovered the course. Can I join now?"]
doc = ["Can I still join the course after the start date?"]

# Generate embeddings
query_embedding = next(embedder.embed(query))
doc_embedding = next(embedder.embed(doc))

# Compute cosine similarity
similarity = cosine_similarity(
    np.array(query_embedding).reshape(1, -1),
    np.array(doc_embedding).reshape(1, -1)
)

print("Cosine similarity:", similarity[0][0])

2025-06-29 12:16:17.186 | WARNING  | fastembed.embedding:<module>:7 - DefaultEmbedding, FlagEmbedding, JinaEmbedding are deprecated.Use from fastembed import TextEmbedding instead.


Cosine similarity: 0.9008528895674549


## Q3. Ranking by cosine

In [5]:
documents = [{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.',
  'section': 'General course-related questions',
  'question': 'Course - Can I follow the course after it finishes?',
  'course': 'data-engineering-zoomcamp'},
 {'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
  'section': 'General course-related questions',
  'question': 'Course - When will the course start?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'You can start by installing and setting up all the dependencies and requirements:\nGoogle cloud account\nGoogle Cloud SDK\nPython 3 (installed with Anaconda)\nTerraform\nGit\nLook over the prerequisites and syllabus to see if you are comfortable with these subjects.',
  'section': 'General course-related questions',
  'question': 'Course - What can I do before the course starts?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Star the repo! Share it with friends if you find it useful ❣️\nCreate a PR if you see you can improve the text or the structure of the repository.',
  'section': 'General course-related questions',
  'question': 'How can we contribute to the course?',
  'course': 'data-engineering-zoomcamp'}]

In [6]:
query = ["I just discovered the course. Can I join now?"]

embedder = TextEmbedding(model_name="jinaai/jina-embeddings-v2-small-en")

query_embedding = next(embedder.embed(query))
doc_embeddings = list(embedder.embed([doc['text'] for doc in documents]))

similarities = cosine_similarity([query_embedding], doc_embeddings)[0]
max_index = np.argmax(similarities)

print("Most similar document index:", max_index)
print("Cosine similarity:", similarities[max_index])

Most similar document index: 1
Cosine similarity: 0.8182378150042889


## Q4. Ranking by cosine, version two

In [7]:
query = ["I just discovered the course. Can I join now?"]

embedder = TextEmbedding(model_name="jinaai/jina-embeddings-v2-small-en")

query_embedding = next(embedder.embed(query))
doc_embeddings = list(embedder.embed([doc['question'] + ' ' + doc['text'] for doc in documents]))

similarities = cosine_similarity([query_embedding], doc_embeddings)[0]
max_index = np.argmax(similarities)

print("Most similar document index:", max_index)
print("Cosine similarity:", similarities[max_index])

Most similar document index: 0
Cosine similarity: 0.8514543236908068


## Q5. Selecting the embedding model

In [9]:
import json

for model in TextEmbedding.list_supported_models():
    if model["model"] == 'BAAI/bge-small-en':
        print(json.dumps(model, indent=2))

{
  "model": "BAAI/bge-small-en",
  "sources": {
    "hf": "Qdrant/bge-small-en",
    "url": "https://storage.googleapis.com/qdrant-fastembed/BAAI-bge-small-en.tar.gz",
    "_deprecated_tar_struct": true
  },
  "model_file": "model_optimized.onnx",
  "description": "Text embeddings, Unimodal (text), English, 512 input tokens truncation, Prefixes for queries/documents: necessary, 2023 year.",
  "license": "mit",
  "size_in_GB": 0.13,
  "additional_files": [],
  "dim": 384,
  "tasks": {}
}


## Q6. Indexing with qdrant (2 points)

In [12]:
from qdrant_client import QdrantClient, models

In [10]:
import requests 

docs_url = 'https://github.com/alexeygrigorev/llm-rag-workshop/raw/main/notebooks/documents.json'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()


documents = []

for course in documents_raw:
    course_name = course['course']
    if course_name != 'machine-learning-zoomcamp':
        continue

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [13]:
model_handle = 'BAAI/bge-small-en'

# Define the collection name
collection_name = "zoomcamp-rag"
EMBEDDING_DIMENSIONALITY = 384

client = QdrantClient("http://localhost:6333")

# Create the collection with specified vector parameters
client.create_collection(
    collection_name=collection_name,
    vectors_config=models.VectorParams(
        size=EMBEDDING_DIMENSIONALITY,  # Dimensionality of the vectors
        distance=models.Distance.COSINE  # Distance metric for similarity search
    )
)

True

In [14]:
points = []
id = 0

for course in documents_raw:
    for doc in course['documents']:

        point = models.PointStruct(
            id=id,
            vector=models.Document(text=doc['text'], model=model_handle), #embed text locally with "jinaai/jina-embeddings-v2-small-en" from FastEmbed
            payload={
                "text": doc['question'] + ' ' + doc['text'],
                "section": doc['section'],
                "course": course['course']
            } #save all needed metadata fields
        )
        points.append(point)

        id += 1

In [15]:
client.upsert(
    collection_name=collection_name,
    points=points
)

Fetching 5 files: 100%|██████████| 5/5 [00:01<00:00,  3.83it/s]


UpdateResult(operation_id=0, status=<UpdateStatus.COMPLETED: 'completed'>)

In [16]:
def search(query, limit=1):

    results = client.query_points(
        collection_name=collection_name,
        query=models.Document( #embed the query text locally with "jinaai/jina-embeddings-v2-small-en"
            text=query,
            model=model_handle 
        ),
        limit=limit, # top closest matches
        with_payload=True #to get metadata in the results
    )

    return results

In [17]:
query = "I just discovered the course. Can I join now?"
result = search(query)
result

QueryResponse(points=[ScoredPoint(id=497, version=0, score=0.8661978, payload={'text': 'Can I use LinearRegression from Scikit-Learn for this week? Yes, you can. We will also do that next week, so don’t worry, you will learn how to do it.', 'section': '2. Machine Learning for Regression', 'course': 'machine-learning-zoomcamp'}, vector=None, shard_key=None, order_value=None)])